## READ ME
- $N(\mathbf{0},I_2)$に従うサンプルを$k$つ生成しその標本分散を$\Sigma$とする．

## ライブラリのimport

In [82]:
# !pip install mip 
from mip import Model, maximize, minimize, xsum
import numpy as np
from numpy import pi
from numpy.random import rand, randn
from numpy.linalg import solve, det
from scipy.linalg import sqrtm
import math


## 定数の設定

In [83]:
p = 2 # 行列の次元
N = 400 # データ数
lam = 0.01 # 正則化パラメータ
delta = 0.8
epsilon = 0.2
h = 3.5 #U(-h,h)
k = 10

## Wasserstein距離を計算する関数

In [84]:
def wasserstein(x, y):
    z = sqrtm(sqrtm(x) @ y @ sqrtm(x))
    res = np.trace(x) + np.trace(y) - 2 * np.trace(z)
    return res

## Entropy正則化されたWasserstein距離を計算する関数

In [85]:
def reg_wasserstein(x, y):
    M_lam = sqrtm(lam * lam * np.eye(p) + sqrtm(x) @ y @ sqrtm(x))
    res = np.trace(x) + np.trace(y) - 2.0 * np.trace(M_lam) - 2.0 * lam * math.log(det(M_lam - lam * np.eye(p)),math.e) - 2.0 * lam * p * math.log(2.0 * math.pi * lam, math.e) - 4.0 * lam * p * math.log(2.0 * math.pi, math.e) - 2.0 * lam * p + 2.0 * lam * math.log(det(x @ y),math.e) + 4.0 * lam * p * (math.log(2.0 * math.pi, math.e) + 1.0)
    return res

In [86]:
## test
# p = 2
print(wasserstein(np.eye(p),np.eye(p)))

0.0


In [87]:
# lam = 10.0
print(reg_wasserstein(np.eye(p),np.eye(p)))

0.1508917231165336


## 乱数の生成と距離グラフの生成

### wasserstein

In [88]:
# samp = randn(10, 1)
# samp = samp.reshape([2,5])
# sigma = np.cov(samp)
# print(sigma)

In [89]:
P = []
nbh_w = [[] for _ in range(N)]

i = 0
np.random.seed(123)
while i < N:
    # ランダム行列vの生成
    samp = randn((2 * k), 1)
    samp = samp.reshape([2,k])
    v = np.cov(samp)        
    # B(I,delta)内になければskip
    if det(v) < 0.0001:
        continue 
    if wasserstein(np.eye(p),v) > delta :
        continue
    
    for j in range(0,i):
        w_dist = wasserstein(P[j],v)
        if w_dist < epsilon:
            nbh_w[i].append(j)
            nbh_w[j].append(i)
            
    # print(i)    
    nbh_w[i].append(i)
            
    i += 1
    P.append(v)

print(nbh_w)
print(len(P),len(nbh_w))
    

[[0, 1, 3, 5, 14, 17, 20, 21, 23, 27, 29, 40, 41, 43, 47, 48, 57, 59, 64, 65, 71, 72, 74, 77, 79, 81, 82, 83, 88, 94, 98, 100, 109, 110, 112, 121, 123, 126, 128, 133, 135, 138, 139, 148, 154, 163, 164, 166, 167, 170, 171, 172, 176, 182, 183, 194, 198, 199, 204, 210, 213, 215, 224, 227, 228, 236, 240, 244, 249, 252, 255, 261, 262, 264, 271, 273, 278, 279, 280, 283, 286, 292, 296, 297, 300, 301, 306, 308, 309, 312, 315, 316, 318, 319, 330, 334, 337, 338, 344, 347, 349, 355, 356, 360, 361, 367, 372, 373, 375, 379, 380, 388, 392, 396, 397, 399], [0, 1, 3, 7, 10, 14, 16, 20, 23, 25, 27, 30, 40, 41, 45, 49, 55, 64, 65, 71, 72, 74, 77, 79, 81, 83, 86, 88, 90, 94, 96, 97, 98, 109, 110, 117, 123, 126, 127, 128, 138, 146, 149, 167, 172, 181, 183, 187, 194, 196, 198, 199, 203, 204, 209, 213, 215, 217, 219, 224, 227, 228, 240, 241, 250, 253, 255, 256, 261, 262, 264, 267, 271, 273, 275, 276, 278, 279, 280, 297, 301, 306, 308, 312, 319, 323, 334, 336, 346, 347, 355, 356, 367, 375, 376, 380, 392, 394

### regularized wasserstein

In [90]:
P = []
nbh_rw = [[] for _ in range(N)]


i = 0
np.random.seed(123)
while i < N:
    # ランダム行列vの生成
    samp = randn((2 * k), 1)
    samp = samp.reshape([2,k])
    v = np.cov(samp)
    # B(I,delta)内になければskip
    if reg_wasserstein(np.eye(p),v) > delta:
        continue
    
    for j in range(0,i):
        rw_dist = reg_wasserstein(P[j],v)
        if rw_dist < epsilon:
            nbh_rw[i].append(j)
            nbh_rw[j].append(i)
            
    # print(i)    
    nbh_rw[i].append(i)
            
    i += 1
    P.append(v)

print(nbh_rw)
print(len(P),len(nbh_rw))
    

[[0, 57, 112, 227, 262, 271, 280, 337, 396], [1, 20, 204, 217, 256, 297, 334, 347, 375], [2, 38, 124, 143, 232, 293, 374], [3, 14, 17, 18, 21, 29, 39, 43, 46, 48, 54, 59, 60, 64, 65, 71, 72, 74, 78, 82, 88, 91, 96, 98, 108, 110, 115, 117, 123, 131, 132, 135, 138, 141, 149, 155, 162, 167, 170, 172, 178, 185, 187, 192, 194, 196, 197, 198, 199, 201, 203, 205, 206, 208, 209, 213, 214, 233, 248, 249, 254, 255, 257, 260, 273, 274, 276, 278, 286, 301, 306, 309, 317, 329, 331, 344, 348, 358, 361, 370, 386, 390, 391, 392, 393], [4, 9, 38, 177, 191, 216, 223, 232, 245, 288, 293, 307, 322, 353, 366, 371, 374, 394], [5, 360], [6, 101, 129, 147, 159, 174, 175, 186, 193, 202, 251, 266, 270, 304, 324, 339, 341, 348, 352, 369, 388, 389], [7, 10, 16, 34, 56, 69, 117, 125, 158, 165, 180, 196, 200, 219, 225, 268, 275, 313, 323, 336, 340, 346], [8, 11, 15, 19, 28, 50, 51, 56, 61, 62, 70, 73, 85, 87, 101, 106, 108, 113, 114, 119, 120, 125, 130, 136, 137, 141, 144, 145, 156, 160, 162, 168, 169, 173, 179, 18

## Covering number

### Wasserstein

In [91]:
from mip import Model, maximize, minimize, xsum

In [92]:
m = Model()  # 数理モデル
x = []
# 変数
for i in range(N):
    s = "x"+str(i)
    x.append(m.add_var(s, lb=0, var_type="I"))

# 目的関数
z = x[0]
for i in range(1,N):
    z += x[i]

m.objective = minimize(z)
# m.objective = maximize(100 * x + 100 * y)
# 制約条件
for i in range(N):
    b = 0
    for j in nbh_w[i]:
        b += x[j]
    m += b >= 1
    
m.optimize()  # ソルバーの実行
print(m.objective.x)

# print("obj", m.objective.x, "x", x.x, "y", y.x)

7.0


### Regularized Wasserstein

In [93]:
m = Model()  # 数理モデル
x = []
# 変数
for i in range(N):
    s = "x"+str(i)
    x.append(m.add_var(s, lb=0, var_type="I"))

# 目的関数
z = x[0]
for i in range(1,N):
    z += x[i]

m.objective = minimize(z)
# m.objective = maximize(100 * x + 100 * y)
# 制約条件
for i in range(N):
    b = 1
    for j in nbh_rw[i]:
        b += x[j]
    m += b >= 2
    
m.optimize()  # ソルバーの実行
print(m.objective.x)

34.0
